In [2]:
# Jupyter Notebook Version of the Resampling Script

# Import necessary libraries
from pathlib import Path
from multiprocessing import Pool
import logging
import pandas as pd
import numpy as np
import SimpleITK as sitk
import click
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
import os
import nibabel as nib


setup input and output folders

In [3]:
# Setting up logger to capture information and warnings
log_fmt = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
logging.basicConfig(level=logging.INFO, format=log_fmt)
logging.captureWarnings(True)
logger = logging.getLogger(__name__)

In [4]:
#  Function to get bounding boxes from CT and PT images
def get_bounding_boxes(ct, pt):
    """
    Get the bounding boxes of the CT and PT images.
    This works since all images have the same direction.
    """
    
    # Get the origins of both CT and PT images
    ct_origin = np.array(ct.GetOrigin())
    pt_origin = np.array(pt.GetOrigin())
    
    # Calculate the maximum positions for both CT and PT images
    ct_position_max = ct_origin + np.array(ct.GetSize()) * np.array(ct.GetSpacing())
    pt_position_max = pt_origin + np.array(pt.GetSize()) * np.array(pt.GetSpacing())
    
    # Return the combined bounding box that covers both images
    return np.concatenate([
        np.maximum(ct_origin, pt_origin),
        np.minimum(ct_position_max, pt_position_max),
    ], axis=0)

In [14]:
# Function to visualize CT data interactively
def show_ct_data(ct_data):
    # Ensure ct_data is a NumPy array
    ct_data = np.asarray(ct_data)
    
    # Define the function to plot a specific slice
    def plot_slice(slice_idx):
        plt.figure(figsize=(6, 6))
        plt.imshow(ct_data[:, :, slice_idx], cmap='gray')
        plt.title(f"Slice {slice_idx}")
        # plt.axis('off')
        plt.show()
    
    # Create an interactive slider to select slices
    interact(plot_slice, slice_idx=widgets.IntSlider(min=0, max=ct_data.shape[2] - 1, step=1, value=0))

In [15]:
# Function to visualize CT data with label contours interactively
def show_ct_data_with_labels(ct_data, label_data):
    """
    Display CT image slices with overlaid label contours interactively.

    Parameters:
    - ct_data: NumPy array of CT image data with shape (height, width, slices).
    - label_data: NumPy array of label data with the same shape as ct_data.
    """
    # Ensure ct_data and label_data are NumPy arrays
    ct_data = np.asarray(ct_data)
    label_data = np.asarray(label_data)
    
    # Verify that the shapes match
    if ct_data.shape != label_data.shape:
        raise ValueError("CT data and label data must have the same shape.")
    
    # Define the function to plot a specific slice
    def plot_slice(slice_idx):
        plt.figure(figsize=(6, 6))
        plt.imshow(ct_data[:, :, slice_idx], cmap='gray')
        
        # Overlay the label as a contour
        contours = plt.contour(label_data[:, :, slice_idx], levels=np.unique(label_data), colors='r', linewidths=0.5)
        
        # Optionally, add a colorbar or legend for labels
        # plt.clabel(contours, inline=True, fontsize=8)
        
        plt.title(f"Slice {slice_idx}")
        plt.axis('off')
        plt.show()
    
    # Create an interactive slider to select slices
    interact(plot_slice, slice_idx=widgets.IntSlider(min=0, max=ct_data.shape[2] - 1, step=1, value=0))

In [17]:
def apply_window(img, width=500, center=40, normal=True):
    img = np.asarray(img)
    # convert below window to black
    img[img<(center-(width/2))]=center-(width/2)
    # convert above window to white
    img[img>(center+(width/2))]=center+(width/2)

    if normal:
        # normalize image
        img_scaled = (img - np.min(img)) / (np.max(img) - np.min(img))
        # print(img_scaled.shape)
        # print(np.min(img_scaled), np.max(img_scaled))
        return img_scaled
    else:
        return img

# Start here

In [71]:
# Define paths for input images, labels, and output directories
# path_input_images = "../dataset/images"
# path_input_labels = "../dataset/labels"
# path_output_images = "../dataset/output/images"
# path_output_labels = "../dataset/output/labels"

path_input_images = "/Volumes/Seagate/Research/ADDA/original/images"
path_input_labels = "/Volumes/Seagate/Research/ADDA/original/labels"
path_output_images = "/Volumes/Seagate/Research/ADDA/resampled_subsets/images"
path_output_labels = "/Volumes/Seagate/Research/ADDA/resampled_subsets/labels"
path_data_breakdown = '../../datasets/ADDA/png_patches_2d/png_patch_training_file_with_tags.csv'

In [72]:
# Set the Paths to the 
input_image_folder = path_input_images
input_label_folder = path_input_labels
output_image_folder = path_output_images
output_label_folder = path_output_labels

cores = 12

resampling = (0.976562, 0.976562, 3.27)

# Resolve paths to absolute paths
input_image_folder = Path(input_image_folder).resolve()
input_label_folder = Path(input_label_folder).resolve()
output_image_folder = Path(output_image_folder).resolve()
output_label_folder = Path(output_label_folder).resolve()


print('Input image folder:', input_image_folder)
print('Input label folder:', input_label_folder)
print('Output image folder:', output_image_folder)
print('Output label folder:', output_label_folder)
print('Resampling voxel size:', resampling)


Input image folder: /Volumes/Seagate/Research/ADDA/original/images
Input label folder: /Volumes/Seagate/Research/ADDA/original/labels
Output image folder: /Volumes/Seagate/Research/ADDA/resampled_subsets/images
Output label folder: /Volumes/Seagate/Research/ADDA/resampled_subsets/labels
Resampling voxel size: (0.976562, 0.976562, 3.27)


## get the list of patient images to resample

1. load from input folder:

In [130]:
# Get list of patients by finding CT images in the input folder
# patient_list = [
#     f.name.split("__")[0] for f in input_image_folder.rglob("*_CT*")
# ]
# if len(patient_list) == 0:
#     raise ValueError("No patient found in the input folder")

# len(patient_list)
# patient_list

['MDA-079', 'CHUM-002', 'CHUM-001', 'MDA-117']

2. load from previous list of CSV file

we need to keep tabs of this file to create a new list later

In [19]:
df = pd.read_csv(path_data_breakdown)
df = df[['subject_id', 'dataset', 'set']]
df.head()

,subject_id,dataset,set
0,MDA-039,source,train
1,MDA-039,source,train
2,MDA-039,source,train
3,MDA-039,source,train
4,MDA-039,source,train


In [20]:
df.groupby(['dataset', 'set']).subject_id.nunique().reset_index()

,dataset,set,subject_id
0,source,test,28
1,source,train,128
2,source,validation,28
3,target,test,15
4,target,train,50
5,target,validation,7


In [21]:
# remove duplicates
df = df[df.duplicated() == False].reset_index(drop=True)
df.head()


,subject_id,dataset,set
0,MDA-039,source,train
1,MDA-192,source,train
2,MDA-187,source,train
3,MDA-179,source,train
4,MDA-162,source,train


In [22]:
df_dims = df.copy()

create a df to show the sizes of each image and their labels for analysis

In [34]:
# extract voxel size for the image
for i, sid in enumerate(df_dims['subject_id']):
#sid = 'MDA-179'
    img_path = os.path.join(input_image_folder, sid+'__CT.nii.gz')
    pt_path = os.path.join(input_image_folder, sid+'__PT.nii.gz')
    lbl_path = os.path.join(input_label_folder, sid+'.nii.gz')

    x = nib.load(img_path)
    x1 = nib.load(pt_path)
    x2 = nib.load(lbl_path)

    # get CT image data
    pixdim_x, pixdim_y, pixdim_z = x.header['pixdim'][1], x.header['pixdim'][2], x.header['pixdim'][3]
    dim_x, dim_y, dim_z = x.header['dim'][1], x.header['dim'][2], x.header['dim'][3]
    # save image dimensions
    df_dims.loc[df_dims['subject_id'] == sid, 'ct_dim_x'] = dim_x
    df_dims.loc[df_dims['subject_id'] == sid, 'ct_dim_y'] = dim_y
    df_dims.loc[df_dims['subject_id'] == sid, 'ct_dim_z'] = dim_z
    # save vovel dimensions
    df_dims.loc[df_dims['subject_id'] == sid, 'ct_pixdim_x'] = pixdim_x
    df_dims.loc[df_dims['subject_id'] == sid, 'ct_pixdim_y'] = pixdim_y
    df_dims.loc[df_dims['subject_id'] == sid, 'ct_pixdim_z'] = pixdim_z

    # Get Pet image dimenstion
    pixdim_x1, pixdim_y1, pixdim_z1 = x1.header['pixdim'][1], x1.header['pixdim'][2], x1.header['pixdim'][3]
    dim_x1, dim_y1, dim_z1 = x1.header['dim'][1], x1.header['dim'][2], x1.header['dim'][3]
    # save image dimensions
    df_dims.loc[df_dims['subject_id'] == sid, 'pt_dim_x'] = dim_x1
    df_dims.loc[df_dims['subject_id'] == sid, 'pt_dim_y'] = dim_y1
    df_dims.loc[df_dims['subject_id'] == sid, 'pt_dim_z'] = dim_z1
    # save vovel dimensions
    df_dims.loc[df_dims['subject_id'] == sid, 'pt_pixdim_x'] = pixdim_x1
    df_dims.loc[df_dims['subject_id'] == sid, 'pt_pixdim_y'] = pixdim_y1
    df_dims.loc[df_dims['subject_id'] == sid, 'pt_pixdim_z'] = pixdim_z1

    # Get Label image dimenstion
    pixdim_x2, pixdim_y2, pixdim_z2 = x2.header['pixdim'][1], x2.header['pixdim'][2], x2.header['pixdim'][3]
    dim_x2, dim_y2, dim_z2 = x2.header['dim'][1], x2.header['dim'][2], x2.header['dim'][3]
    # save image dimensions
    df_dims.loc[df_dims['subject_id'] == sid, 'lbl_dim_x'] = dim_x2
    df_dims.loc[df_dims['subject_id'] == sid, 'lbl_dim_y'] = dim_y2
    df_dims.loc[df_dims['subject_id'] == sid, 'lbl_dim_z'] = dim_z2
    # save vovel dimensions
    df_dims.loc[df_dims['subject_id'] == sid, 'lbl_pixdim_x'] = pixdim_x2
    df_dims.loc[df_dims['subject_id'] == sid, 'lbl_pixdim_y'] = pixdim_y2
    df_dims.loc[df_dims['subject_id'] == sid, 'lbl_pixdim_z'] = pixdim_z2

    # print counter 
    if i % 10 == 0:
        percentage = (i / len(df_dims['subject_id'])) * 100
        print(f'Round {i}: {percentage:.2f}% completed')

df_dims.head()


Round 0: 0.00% completed
Round 10: 3.91% completed
Round 20: 7.81% completed
Round 30: 11.72% completed
Round 40: 15.62% completed
Round 50: 19.53% completed
Round 60: 23.44% completed
Round 70: 27.34% completed
Round 80: 31.25% completed
Round 90: 35.16% completed
Round 100: 39.06% completed
Round 110: 42.97% completed
Round 120: 46.88% completed
Round 130: 50.78% completed
Round 140: 54.69% completed
Round 150: 58.59% completed
Round 160: 62.50% completed
Round 170: 66.41% completed
Round 180: 70.31% completed
Round 190: 74.22% completed
Round 200: 78.12% completed
Round 210: 82.03% completed
Round 220: 85.94% completed
Round 230: 89.84% completed
Round 240: 93.75% completed
Round 250: 97.66% completed


,subject_id,dataset,set,ct_dim_x,ct_dim_y,ct_dim_z,ct_pixdim_x,ct_pixdim_y,ct_pixdim_z,pt_dim_x,...,pt_dim_z,pt_pixdim_x,pt_pixdim_y,pt_pixdim_z,lbl_dim_x,lbl_dim_y,lbl_dim_z,lbl_pixdim_x,lbl_pixdim_y,lbl_pixdim_z
0,MDA-039,source,train,512.0,512.0,127.0,0.976562,0.976562,3.27,128.0,...,127.0,5.46875,5.46875,3.27,512.0,512.0,127.0,0.976562,0.976562,3.27
1,MDA-192,source,train,512.0,512.0,127.0,0.976562,0.976562,3.27,128.0,...,127.0,5.46875,5.46875,3.27,512.0,512.0,127.0,0.976562,0.976562,3.27
2,MDA-187,source,train,512.0,512.0,127.0,0.976562,0.976562,3.27,128.0,...,127.0,5.46875,5.46875,3.27,512.0,512.0,127.0,0.976562,0.976562,3.27
3,MDA-179,source,train,512.0,512.0,127.0,0.976562,0.976562,3.27,128.0,...,127.0,5.46875,5.46875,3.27,512.0,512.0,127.0,0.976562,0.976562,3.27
4,MDA-162,source,train,512.0,512.0,87.0,0.976562,0.976562,3.27,128.0,...,87.0,5.46875,5.46875,3.27,512.0,512.0,87.0,0.976562,0.976562,3.27


In [44]:
dim_cols = [
    'ct_dim_x', 'ct_dim_y',
    'pt_dim_x', 'pt_dim_y', 
    'lbl_dim_x', 'lbl_dim_y',
]

# Check if all rows have the same values across the selected columns
same_dims = df_dims[dim_cols].nunique(axis=1) == 1

# Output rows where dimensions are not the same (False means the dimensions vary in those rows)
inconsistent_rows = df_dims[~same_dims]

# Print the number of inconsistent rows and inspect them
print(f"Number of rows with inconsistent dimensions: {len(inconsistent_rows)}")
print(inconsistent_rows['subject_id'])


Number of rows with inconsistent dimensions: 256
0       MDA-039
1       MDA-192
2       MDA-187
3       MDA-179
4       MDA-162
         ...   
251    CHUP-027
252    CHUP-030
253    CHUP-038
254    CHUP-052
255    CHUP-061
Name: subject_id, Length: 256, dtype: object


In [48]:
# find which pet iimages are not 128

df_dims[df_dims['pt_dim_x'] != 128]


,subject_id,dataset,set,ct_dim_x,ct_dim_y,ct_dim_z,ct_pixdim_x,ct_pixdim_y,ct_pixdim_z,pt_dim_x,...,pt_dim_z,pt_pixdim_x,pt_pixdim_y,pt_pixdim_z,lbl_dim_x,lbl_dim_y,lbl_dim_z,lbl_pixdim_x,lbl_pixdim_y,lbl_pixdim_z
13,MDA-113,source,train,512.0,512.0,134.0,0.976562,0.976562,2.80,256.0,...,134.0,2.734375,2.734375,2.800,512.0,512.0,134.0,0.976562,0.976562,2.80
21,MDA-120,source,train,512.0,512.0,407.0,0.976562,0.976562,3.27,192.0,...,407.0,3.645833,3.645833,3.270,512.0,512.0,408.0,0.976562,0.976562,3.27
39,MDA-114,source,train,512.0,512.0,119.0,0.976562,0.976562,3.27,192.0,...,119.0,3.645833,3.645833,3.270,512.0,512.0,119.0,0.976562,0.976562,3.27
51,MDA-112,source,train,512.0,512.0,119.0,0.976562,0.976562,3.27,256.0,...,119.0,2.734375,2.734375,3.270,512.0,512.0,119.0,0.976562,0.976562,3.27
53,MDA-117,source,train,512.0,512.0,335.0,0.976562,0.976562,3.27,192.0,...,335.0,3.645833,3.645833,3.270,512.0,512.0,335.0,0.976562,0.976562,3.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,CHUP-027,target,validation,512.0,512.0,736.0,0.976562,0.976562,1.50,200.0,...,543.0,4.072830,4.072830,2.027,512.0,512.0,736.0,0.976562,0.976562,1.50
252,CHUP-030,target,validation,512.0,512.0,736.0,0.976562,0.976562,1.50,200.0,...,543.0,4.072830,4.072830,2.027,512.0,512.0,736.0,0.976562,0.976562,1.50
253,CHUP-038,target,validation,512.0,512.0,568.0,0.976562,0.976562,1.50,200.0,...,419.0,4.072830,4.072830,2.027,512.0,512.0,568.0,0.976562,0.976562,1.50
254,CHUP-052,target,validation,512.0,512.0,568.0,0.976562,0.976562,1.50,200.0,...,419.0,4.072830,4.072830,2.027,512.0,512.0,568.0,0.976562,0.976562,1.50


In [49]:
df_dims[df_dims['subject_id']=='CHUP-030']

,subject_id,dataset,set,ct_dim_x,ct_dim_y,ct_dim_z,ct_pixdim_x,ct_pixdim_y,ct_pixdim_z,pt_dim_x,...,pt_dim_z,pt_pixdim_x,pt_pixdim_y,pt_pixdim_z,lbl_dim_x,lbl_dim_y,lbl_dim_z,lbl_pixdim_x,lbl_pixdim_y,lbl_pixdim_z
252,CHUP-030,target,validation,512.0,512.0,736.0,0.976562,0.976562,1.5,200.0,...,543.0,4.07283,4.07283,2.027,512.0,512.0,736.0,0.976562,0.976562,1.5


## Start resampling

In [73]:
# Main function to perform resampling
def main(patient, input_image_folder, input_label_folder, output_image_folder,
         output_label_folder, cores, resampling):
    """Resamples NIFTI files within the maximal bounding box covered by both modalities (PT and CT)."""
    
    logger.info('Resampling')

    # Resolve paths to absolute paths
    # input_image_folder = Path(input_image_folder).resolve()
    # input_label_folder = Path(input_label_folder).resolve()
    # output_image_folder = Path(output_image_folder).resolve()
    # output_label_folder = Path(output_label_folder).resolve()

    # Create output folders if they do not exist
    # output_image_folder.mkdir(exist_ok=True)
    # output_label_folder.mkdir(exist_ok=True)
    # print('Resampling with voxel size: {}'.format(resampling))

    # Get list of patients by finding CT images in the input folder
    # patient_list = [
    #     f.name.split("__")[0] for f in input_image_folder.rglob("*_CT*")
    # ]
    # if len(patient_list) == 0:
    #     raise ValueError("No patient found in the input folder")

    # print(patient_list)
    
    # Initialize the resampler with default settings
    resampler = sitk.ResampleImageFilter()
    resampler.SetOutputDirection([1, 0, 0, 0, 1, 0, 0, 0, 1])
    resampler.SetOutputSpacing(resampling)

    # Define the function to resample one patient's images and labels
    def resample_one_patient(p):
        # Read CT and PT images for the patient
        ct = sitk.ReadImage(str([f for f in input_image_folder.rglob(p + "__CT*")][0]))
        pt = sitk.ReadImage(str([f for f in input_image_folder.rglob(p + "__PT*")][0]))
        
        # Read label images for the patient
        labels = [(sitk.ReadImage(str(f)), f.name) for f in input_label_folder.glob(p + "*")]
        
        # Get bounding box for the resampling operation
        bb = get_bounding_boxes(ct, pt)
        size = np.round((bb[3:] - bb[:3]) / resampling).astype(int)
        
        # Configure resampler settings for CT and PT images
        resampler.SetOutputOrigin(bb[:3])
        resampler.SetSize([int(k) for k in size])
        resampler.SetInterpolator(sitk.sitkBSpline)  # Use cubic spline interpolation for CT and PT
        
        # Resample CT and PT images
        ct = resampler.Execute(ct)
        pt = resampler.Execute(pt)
        
        # # # Save the resampled CT and PT images to the output folder
        # sitk.WriteImage(ct, str((output_image_folder / (p + "__CT.nii.gz"))))
        # sitk.WriteImage(pt, str((output_image_folder / (p + "__PT.nii.gz"))))
        
        # Configure resampler for label images using nearest neighbor interpolation
        resampler.SetInterpolator(sitk.sitkNearestNeighbor)
        for label, name in labels:
            label = resampler.Execute(label)
            # sitk.WriteImage(label, str((output_label_folder / name)))
        
        return ct, pt, label

    # # Loop through each patient and resample their images and labels
    # for p in patient_list:
    #     resample_one_patient(p)
    # # Uncomment for parallel processing to speed up the operation
    # # with Pool(cores) as p:
    # #     p.map(resample_one_patient, patient_list)

    ct_res, pet_res, lbl_res = resample_one_patient(patient)
    return ct_res, pet_res, lbl_res


In [66]:
def process_patient(patient, input_image_folder, input_label_folder, output_image_folder, output_label_folder, cores, resampling):
    """
    Processes a single patient's CT, PET, and label images, applies a window function, 
    and rotates them 90 degrees.
    
    Args:
    - patient (str): Identifier for the patient to process.
    - input_image_folder (str): Path to the folder containing the input images.
    - input_label_folder (str): Path to the folder containing the input labels.
    - output_image_folder (str): Path to the folder where processed images will be saved.
    - output_label_folder (str): Path to the folder where processed labels will be saved.
    - cores (int): Number of cores to use for processing.
    - resampling (str): Resampling parameter.

    Returns:
    - (tuple): Rotated and processed CT, PET, and label arrays.
    """
    # Call the main function to start resampling (one patient)
    c, p, l = main(patient, input_image_folder, input_label_folder, output_image_folder, output_label_folder, cores, resampling)

    # Convert images to NumPy arrays and apply windowing to CT image
    cn = apply_window(sitk.GetArrayFromImage(c))
    pn = sitk.GetArrayFromImage(p)
    ln = sitk.GetArrayFromImage(l)
    
    # Print the shapes for verification
    print(f"Patient: {patient}")
    print("Shapes before rotation:", cn.shape, pn.shape, ln.shape)

    # Rotate the images by 90 degrees along (0, 2) axes
    pn_r = np.rot90(pn, axes=(0, 2))
    cn_r = np.rot90(cn, axes=(0, 2))
    ln_r = np.rot90(ln, axes=(0, 2))

    # Print the shapes after rotation for verification
    print("Shapes after rotation:", cn_r.shape, pn_r.shape, ln_r.shape)
    
    # Return the rotated images
    return cn_r, pn_r, ln_r



In [77]:
for i, sid in enumerate(df_dims['subject_id']):
    # Process patient and get rotated images
    cn_r, pn_r, ln_r = process_patient(sid, input_image_folder, input_label_folder, output_image_folder, output_label_folder, cores, resampling)

    # optional 
    # show_ct_data_with_labels(cn_r, ln_r)
    
    # Convert rotated NumPy arrays back to SimpleITK Image objects
    cn_r_img = sitk.GetImageFromArray(cn_r)
    pn_r_img = sitk.GetImageFromArray(pn_r)
    ln_r_img = sitk.GetImageFromArray(ln_r)

    # Save the resampled CT and PT images to the output folder
    sitk.WriteImage(cn_r_img, str(output_image_folder / (sid + "__CT.nii.gz")))
    sitk.WriteImage(pn_r_img, str(output_image_folder / (sid + "__PT.nii.gz")))

    # Save the resampled Label to the output folder
    sitk.WriteImage(ln_r_img, str(output_label_folder / (sid + ".nii.gz")))

    # Print counter to show progress every 10 iterations
    if i % 10 == 0:
        percentage = (i / len(df_dims['subject_id'])) * 100
        print(f'Round {i}: {percentage:.2f}% completed')

    # Optional: break to limit the loop 
    # break


2024-11-09 11:59:07,650 - __main__ - INFO - Resampling


Patient: MDA-039
Shapes before rotation: (127, 512, 512) (127, 512, 512) (127, 512, 512)
Shapes after rotation: (512, 512, 127) (512, 512, 127) (512, 512, 127)


2024-11-09 11:59:16,766 - __main__ - INFO - Resampling


Round 0: 0.00% completed
Patient: MDA-192
Shapes before rotation: (127, 512, 512) (127, 512, 512) (127, 512, 512)
Shapes after rotation: (512, 512, 127) (512, 512, 127) (512, 512, 127)


2024-11-09 11:59:25,375 - __main__ - INFO - Resampling


...

Patient: CHUP-038
Shapes before rotation: (260, 512, 512) (260, 512, 512) (260, 512, 512)
Shapes after rotation: (512, 512, 260) (512, 512, 260) (512, 512, 260)


2024-11-09 13:03:48,828 - __main__ - INFO - Resampling


Patient: CHUP-052
Shapes before rotation: (260, 512, 512) (260, 512, 512) (260, 512, 512)
Shapes after rotation: (512, 512, 260) (512, 512, 260) (512, 512, 260)


2024-11-09 13:04:11,894 - __main__ - INFO - Resampling


Patient: CHUP-061
Shapes before rotation: (298, 512, 512) (298, 512, 512) (298, 512, 512)
Shapes after rotation: (512, 512, 298) (512, 512, 298) (512, 512, 298)


# Sandbox

In [56]:
show_ct_data(apply_window(x.get_fdata()))


interactive(children=(IntSlider(value=0, description='slice_idx', max=735), Output()), _dom_classes=('widget-i…

In [52]:
# Call the main function to start resampling (one patient)
patient = 'CHUP-030' # patient_list[3]
print(patient)
c, p, l = main(patient, input_image_folder, input_label_folder, output_image_folder, output_label_folder, cores, resampling)

2024-11-09 11:21:23,344 - __main__ - INFO - Resampling


CHUP-030


In [62]:
cn = apply_window(sitk.GetArrayFromImage(c))
pn = sitk.GetArrayFromImage(p)
ln = sitk.GetArrayFromImage(l)
print(cn.shape, pn.shape, ln.shape)

(337, 512, 512) (337, 512, 512) (337, 512, 512)


In [63]:
pn_r = np.rot90(pn, axes=(0,2))
cn_r = np.rot90(cn, axes=(0,2))
ln_r = np.rot90(ln, axes=(0,2))

In [64]:
show_ct_data_with_labels(cn_r, ln_r)

interactive(children=(IntSlider(value=0, description='slice_idx', max=336), Output()), _dom_classes=('widget-i…

In [56]:
show_ct_data_with_labels(pn, ln)

interactive(children=(IntSlider(value=0, description='slice_idx', max=460), Output()), _dom_classes=('widget-i…

In [87]:
import pandas as pd

In [88]:
df = pd.read_csv('../../datasets/ADDA/png_patches_2d/png_patch_training_file_with_tags.csv')
df.head()

,subject_id,dataset,set,ct_path,pet_path,label_path,not_black_frame,tumor
0,MDA-039,source,train,../datasets/ADDA/png_patches_2d/images/MDA-039...,../datasets/ADDA/png_patches_2d/images/MDA-039...,../datasets/ADDA/png_patches_2d/labels/MDA-039...,0,0
1,MDA-039,source,train,../datasets/ADDA/png_patches_2d/images/MDA-039...,../datasets/ADDA/png_patches_2d/images/MDA-039...,../datasets/ADDA/png_patches_2d/labels/MDA-039...,0,0
2,MDA-039,source,train,../datasets/ADDA/png_patches_2d/images/MDA-039...,../datasets/ADDA/png_patches_2d/images/MDA-039...,../datasets/ADDA/png_patches_2d/labels/MDA-039...,0,0
3,MDA-039,source,train,../datasets/ADDA/png_patches_2d/images/MDA-039...,../datasets/ADDA/png_patches_2d/images/MDA-039...,../datasets/ADDA/png_patches_2d/labels/MDA-039...,0,0
4,MDA-039,source,train,../datasets/ADDA/png_patches_2d/images/MDA-039...,../datasets/ADDA/png_patches_2d/images/MDA-039...,../datasets/ADDA/png_patches_2d/labels/MDA-039...,0,0


In [92]:
df['dataset'].unique()

array(['source', 'target'], dtype=object)

In [95]:
df[df['set'] == 'train']['subject_id'].unique()

array(['MDA-039', 'MDA-192', 'MDA-187', 'MDA-179', 'MDA-162', 'MDA-097',
       'MDA-194', 'MDA-095', 'MDA-038', 'MDA-147', 'MDA-150', 'MDA-133',
       'MDA-071', 'MDA-113', 'MDA-023', 'MDA-024', 'MDA-013', 'MDA-059',
       'MDA-006', 'MDA-101', 'MDA-065', 'MDA-120', 'MDA-163', 'MDA-064',
       'MDA-025', 'MDA-088', 'MDA-098', 'MDA-099', 'MDA-078', 'MDA-020',
       'MDA-100', 'MDA-021', 'MDA-005', 'MDA-047', 'MDA-173', 'MDA-146',
       'MDA-068', 'MDA-053', 'MDA-004', 'MDA-114', 'MDA-055', 'MDA-061',
       'MDA-132', 'MDA-164', 'MDA-159', 'MDA-172', 'MDA-143', 'MDA-175',
       'MDA-076', 'MDA-139', 'MDA-018', 'MDA-112', 'MDA-048', 'MDA-117',
       'MDA-043', 'MDA-169', 'MDA-050', 'MDA-190', 'MDA-049', 'MDA-091',
       'MDA-045', 'MDA-067', 'MDA-058', 'MDA-134', 'MDA-082', 'MDA-148',
       'MDA-027', 'MDA-011', 'MDA-121', 'MDA-144', 'MDA-156', 'MDA-184',
       'MDA-183', 'MDA-093', 'MDA-016', 'MDA-092', 'MDA-116', 'MDA-034',
       'MDA-155', 'MDA-108', 'MDA-044', 'MDA-083', 

In [ ]:

# Define paths for input images, labels, and output directories
path_input_images = "../datasets/ADDA/original/images"
path_input_labels = "../datasets/ADDA/original/labels"
path_output_images = "../datasets/ADDA/resampled/images"
path_output_labels = "../datasets/ADDA/resampled/labels"

# Setting up logger to capture information and warnings
log_fmt = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
logging.basicConfig(level=logging.INFO, format=log_fmt)
logging.captureWarnings(True)
logger = logging.getLogger(__name__)

# Function to get bounding boxes from CT and PT images
def get_bounding_boxes(ct, pt):
    """
    Get the bounding boxes of the CT and PT images.
    This works since all images have the same direction.
    """
    
    # Get the origins of both CT and PT images
    ct_origin = np.array(ct.GetOrigin())
    pt_origin = np.array(pt.GetOrigin())
    
    # Calculate the maximum positions for both CT and PT images
    ct_position_max = ct_origin + np.array(ct.GetSize()) * np.array(ct.GetSpacing())
    pt_position_max = pt_origin + np.array(pt.GetSize()) * np.array(pt.GetSpacing())
    
    # Return the combined bounding box that covers both images
    return np.concatenate([
        np.maximum(ct_origin, pt_origin),
        np.minimum(ct_position_max, pt_position_max),
    ], axis=0)

# Main function to perform resampling
def main(input_image_folder, input_label_folder, output_image_folder,
         output_label_folder, cores, resampling):
    """Resamples NIFTI files within the maximal bounding box covered by both modalities (PT and CT)."""
    
    logger.info('Resampling')

    # Resolve paths to absolute paths
    input_image_folder = Path(input_image_folder).resolve()
    input_label_folder = Path(input_label_folder).resolve()
    output_image_folder = Path(output_image_folder).resolve()
    output_label_folder = Path(output_label_folder).resolve()

    # Create output folders if they do not exist
    output_image_folder.mkdir(exist_ok=True)
    output_label_folder.mkdir(exist_ok=True)
    print('Resampling with voxel size: {}'.format(resampling))

    # Get list of patients by finding CT images in the input folder
    patient_list = [
        f.name.split("__")[0] for f in input_image_folder.rglob("*_CT*")
    ]
    if len(patient_list) == 0:
        raise ValueError("No patient found in the input folder")

    # Initialize the resampler with default settings
    resampler = sitk.ResampleImageFilter()
    resampler.SetOutputDirection([1, 0, 0, 0, 1, 0, 0, 0, 1])
    resampler.SetOutputSpacing(resampling)

    # Define the function to resample one patient's images and labels
    def resample_one_patient(p):
        # Read CT and PT images for the patient
        ct = sitk.ReadImage(str([f for f in input_image_folder.rglob(p + "__CT*")][0]))
        pt = sitk.ReadImage(str([f for f in input_image_folder.rglob(p + "__PT*")][0]))
        
        # Read label images for the patient
        labels = [(sitk.ReadImage(str(f)), f.name) for f in input_label_folder.glob(p + "*")]
        
        # Get bounding box for the resampling operation
        bb = get_bounding_boxes(ct, pt)
        size = np.round((bb[3:] - bb[:3]) / resampling).astype(int)
        
        # Configure resampler settings for CT and PT images
        resampler.SetOutputOrigin(bb[:3])
        resampler.SetSize([int(k) for k in size])
        resampler.SetInterpolator(sitk.sitkBSpline)  # Use cubic spline interpolation for CT and PT
        
        # Resample CT and PT images
        ct = resampler.Execute(ct)
        pt = resampler.Execute(pt)
        
        # Save the resampled CT and PT images to the output folder
        sitk.WriteImage(ct, str((output_image_folder / (p + "__CT.nii.gz"))))
        sitk.WriteImage(pt, str((output_image_folder / (p + "__PT.nii.gz"))))
        
        # Configure resampler for label images using nearest neighbor interpolation
        resampler.SetInterpolator(sitk.sitkNearestNeighbor)
        for label, name in labels:
            label = resampler.Execute(label)
            sitk.WriteImage(label, str((output_label_folder / name)))

    # Loop through each patient and resample their images and labels
    for p in patient_list:
        resample_one_patient(p)
    # Uncomment for parallel processing to speed up the operation
    # with Pool(cores) as p:
    #     p.map(resample_one_patient, patient_list)

# Execute the main function with example parameters
input_image_folder = path_input_images
input_label_folder = path_input_labels
output_image_folder = path_output_images
output_label_folder = path_output_labels
cores = 12
resampling = (2, 2, 2)

# Call the main function to start resampling
main(input_image_folder, input_label_folder, output_image_folder, output_label_folder, cores, resampling)